In [1]:
import numpy as np
import pandas as pd
import swifter

import yaml
import importlib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from os import listdir
from collections import Counter, defaultdict

import get_emotes
importlib.reload(get_emotes)
import streaming_chat
importlib.reload(streaming_chat)
import constants

In [2]:
bttv = f'{constants.extensions_path}/ajopnjidmegmdimjlfnijceegpefgped/7.2.80_0'
ff = f'{constants.extensions_path}/fadndhdgpmmaapbmfcknlfgcflmmmieb/1.57_0'
ublock= f'{constants.extensions_path}/cjpalhdlnbpafiamejdnhcphjbkeiagm/1.27.10_0'

chrome_driver = constants.chrome_driver_path
chrome_options = Options()
chrome_options.add_argument(f'--load-extension={ff},{bttv},{ublock}')

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

In [5]:
twitch_driver = get_emotes.TwitchSetup(driver)
twitch_driver.login("ScrapingIsFun", "AutoScraperDSE")
twitch_driver.adjust_settings()

In [2]:
with open("emotes.yaml", 'r') as stream:
    emotes = yaml.safe_load(stream)

In [40]:
emote_list = []
for channel in emotes.keys():
    for emote_type in emotes[channel].keys():
        for emote in emotes[channel][emote_type]:
            emote_list.append({'name': emote, 'emote_type': emote_type, 'channel': channel})

In [147]:
emote_df = pd.DataFrame(emote_list)

In [148]:
emote_channel_df = emote_df[['name', 'channel']].copy()

In [149]:
emote_df = emote_df.drop('channel', axis=1).drop_duplicates('name').set_index('name')

In [150]:
len(emote_df)

1092

In [86]:
chat_logs = listdir('./Chat Logs/')
chat_logs = ['./Chat Logs/' + file for file in chat_logs]
data = streaming_chat.combine_chat_logs(chat_logs)  

In [87]:
df = pd.DataFrame(data)

In [88]:
chat_logs = listdir('./Chat Logs/')
streamers = set()
for log in chat_logs:
    streamers.add(log.split('.log')[0])

In [89]:
df.loc[df.message.apply(len) == 0, 'message'] = np.NaN

df.loc[df['channel'].apply(lambda row: row not in streamers), 'message'] = np.NaN

df.dropna(inplace=True)

In [ ]:
def emote_count2(message):
    emote_freq = defaultdict(int)
    message_data = message.split(' ')
    for word in message_data:
        is_emote = word in emote_df.index
        if is_emote:
            emote_freq[word] += 1
    return Counter(emote_freq)

In [ ]:
df['msg_emotes'] = df.swifter.apply(lambda row: emote_count2(row.message), axis=1)

In [ ]:
count_df = pd.DataFrame(index = emote_count.keys(), data=emote_count.values())
emote_df = pd.merge(left=emote_df, right=count_df, how='left', left_index=True, right_index=True)
emote_df.rename(columns={0: "emote_count"}, inplace=True)

In [ ]:
emote_freq_channel = df.groupby('channel').msg_emotes.sum()
pd.merge(left=emote_df, right=pd.Series(emote_freq_channel['fedmyster']).rename('fedmyster_count'), how='left',left_index=True, right_index=True)

for key in emote_freq_channel.keys():
    emote_df = pd.merge(left=emote_df, right=pd.Series(emote_freq_channel[key]).rename(f'{key}_count'),
                        how='left', left_index=True, right_index=True)

In [226]:
total_messages = df.groupby('channel').message.count()
total_messages

channel
fedmyster     41553
lilypichu     41753
pokimane      44176
scarra        43619
sykkuno       42348
xchocobars    43475
yassuo        41413
yvonnie        8381
Name: message, dtype: int64

In [87]:
x = df.groupby(['channel', 'username'])['message'].count().sort_values().groupby(level=0).tail(5)
x['fedmyster'].index.values

array(['spirit56', 'damstream3', 'fk148', 'izagamii', 'faerydaeflo'],
      dtype=object)

In [225]:
df['is_@_streamer'] = df.apply(lambda row: row['message'].lower().startswith(f"@{row['channel']}"), axis = 1)

In [14]:
df.loc[df['is_@_streamer'] == True].groupby('channel').message.count()

channel
fedmyster      126
lilypichu      318
pokimane       401
scarra        1070
sykkuno        496
xchocobars     642
yassuo        1817
yvonnie         73
Name: message, dtype: int64

In [13]:
num_messages[['yvonnie', 'fedmyster']]

channel
yvonnie       73
fedmyster    126
Name: message, dtype: int64

In [7]:
df['question_mark_messages'] = df.apply(lambda row: row['message'].startswith('?'), axis = 1)

KeyboardInterrupt: 

In [229]:
df.loc[df['question_mark_messages'] == True].groupby('channel').message.count().sort_values()

channel
yvonnie        12
lilypichu      65
xchocobars    193
scarra        209
sykkuno       258
yassuo        364
pokimane      542
fedmyster     584
Name: message, dtype: int64

In [230]:
df['command_message'] = df.apply(lambda row: row['message'].startswith('!'), axis = 1)

In [27]:
df.loc[df['command_message'] == True].groupby('channel').message.count().sort_values()

channel
fedmyster       98
xchocobars     103
yvonnie        182
sykkuno        415
pokimane       723
lilypichu      918
scarra         950
yassuo        1530
Name: message, dtype: int64

In [28]:
command_count_per_channel = df.loc[df['command_message'] == True].groupby(['channel','message'])['username'].count()
top_commands_per_channel = command_count_per_channel.sort_values().groupby(level=0).tail(3).sort_index()
top_commands_per_channel

channel     message   
fedmyster   !discord        5
            !subs           5
            !uptime        47
lilypichu   !donate       113
            !team          61
            !uptime       260
pokimane    !followage     90
            !shaders      164
            !uptime        81
scarra      !rank          84
            !squad        142
            !uptime       101
sykkuno     !duo          109
            !shaders       31
            !uptime       149
xchocobars  !song          11
            !tourney       12
            !uptime        19
yassuo      !charity      240
            !rank         204
            !uptime       280
yvonnie     !charity       84
            !uptime        34
            !watchtime      9
Name: username, dtype: int64

In [231]:
df['message_length'] = df.message.swifter.apply(lambda msg: len(msg.split(' ')))

C:\Users\crsar\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [232]:
df.groupby('channel')['message_length'].median()

channel
fedmyster     1
lilypichu     2
pokimane      2
scarra        3
sykkuno       2
xchocobars    3
yassuo        2
yvonnie       3
Name: message_length, dtype: int64

In [224]:
emote_df['xchocobars_count'].sort_values(ascending=False).head(5)

name
Pog      1501.0
LUL      1374.0
LULW      813.0
YEP       568.0
PepoG     371.0
Name: xchocobars_count, dtype: float64

In [168]:
t1 = pd.merge(emote_df.loc[emote_df.emote_type == 'Tier 1'], emote_channel_df, left_index=True, right_on = 'name')

NameError: name 'emote_df' is not defined

In [201]:
t1.groupby('channel').sum().drop('emote_count', axis = 1)

,fedmyster_count,lilypichu_count,pokimane_count,scarra_count,sykkuno_count,xchocobars_count,yassuo_count,yvonnie_count
channel,,,,,,,,
fedmyster,37379.0,593.0,277.0,225.0,442.0,15.0,2.0,43.0
lilypichu,518.0,7568.0,126.0,166.0,140.0,51.0,0.0,11.0
pokimane,83.0,119.0,3235.0,20.0,7.0,12.0,4.0,2.0
scarra,17.0,67.0,20.0,1351.0,6.0,0.0,0.0,0.0
sykkuno,374.0,401.0,50.0,55.0,10611.0,35.0,2.0,62.0
xchocobars,0.0,37.0,13.0,4.0,0.0,1377.0,1.0,2.0
yassuo,7.0,1.0,28.0,6.0,5.0,0.0,2742.0,0.0
yvonnie,55.0,66.0,15.0,23.0,25.0,8.0,0.0,1652.0


In [233]:
df['num_emotes'] = df.msg_emotes.swifter.apply(lambda x: sum(x.values()))

In [234]:
df['emote_ratio'] = df['num_emotes'] / df['message_length']

In [235]:
df

,time,channel,username,message,msg_emotes,is_@_streamer,question_mark_messages,command_message,message_length,num_emotes,emote_ratio
0,2020-06-04 22:34:00,fedmyster,uplandace12,🥵🥵,{},False,False,False,1,0,0.000000
1,2020-06-04 22:34:00,fedmyster,impmeta,fed7 fed7 fed7 fed7 fed7 fed7 fed7 fed7 fed7 f...,{'fed7': 22},False,False,False,22,22,1.000000
2,2020-06-04 22:34:00,fedmyster,tiger662,fedL fedL fedL fedL fedL fedL fedL,{'fedL': 7},False,False,False,7,7,1.000000
3,2020-06-04 22:34:00,fedmyster,thatoneasianboye,THE MAN THE MYTH THE LEGENDS IS BACKKK,{},False,False,False,8,0,0.000000
4,2020-06-04 22:34:01,fedmyster,liberianrefugee,ppp,{},False,False,False,1,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
309104,2020-06-05 00:50:43,yvonnie,tv_timeout,i don't think it's your call at that point,{},False,False,False,9,0,0.000000
309105,2020-06-05 00:50:43,yvonnie,luigi99212,FEEL THE SAME THEY FELT KEKW,{'KEKW': 1},False,False,False,6,1,0.166667
309106,2020-06-05 00:50:43,yvonnie,snowtheo,no she worked hard on it,{},False,False,False,6,0,0.000000
309107,2020-06-05 00:50:45,yvonnie,mcboi9,cmon yvonne content!,{},False,False,False,3,0,0.000000


In [20]:
df.groupby('channel')['emote_ratio'].mean()[['fedmyster', 'lilypichu']]

channel
fedmyster    0.434949
lilypichu    0.225278
Name: emote_ratio, dtype: float64

In [236]:
def filter_words(msg, msg_emotes):
    ret = []
    for word in msg.split(' '):
        if word not in msg_emotes:
            ret.append(word)
    return ' '.join(ret)

In [237]:
df['message_no_emotes'] = df.swifter.apply(lambda row: filter_words(row['message'], row['msg_emotes']), axis=1)

In [24]:
emote.emote_count.nlargest(20)

name
KEKW          14656.0
fedKING       10902.0
Pog            9755.0
fed7           9343.0
LUL            7723.0
fedL           6021.0
<3             4904.0
Clap           4189.0
YEP            3951.0
LULW           3860.0
PogU           3547.0
sykHeart       3530.0
PogChamp       2884.0
Kappa          2482.0
D:             2310.0
PepeLaugh      2163.0
OMEGALUL       2042.0
EZ             1915.0
monkaW         1828.0
WeirdChamp     1797.0
Name: emote_count, dtype: float64

In [245]:
emote_channel_df.to_pickle('emote_channel.pkl')

In [247]:
emote_df.to_pickle('emote.pkl')

In [3]:
df = pd.read_msgpack('./Data/chat_data.msg')

In [4]:
df.to_pickle('chat_data.pkl')

In [4]:
df = pd.read_pickle('./Data/chat_data.pkl')

In [5]:
emote = pd.read_pickle('./Data/emote.pkl')

In [6]:
emote_channel_df = pd.read_pickle('./Data/emote_channel.pkl')

In [496]:
emote_channel_df

,name,channel
0,gachiGASM,fedmyster
1,HYPERS,fedmyster
2,peepoRiot,fedmyster
3,coffinPls,fedmyster
4,blobDance,fedmyster
...,...,...
1474,yvonWeeb,yvonnie
1475,yvonQ,yvonnie
1476,yvonCry,yvonnie
1477,yvonSadd,yvonnie


In [130]:
emote['modified_emote_type'] = emote['emote_type']

In [131]:
emote.loc[emote['modified_emote_type'].apply(lambda x : x.startswith("FrankerFaceZ")), 'modified_emote_type'] = 'FrankerFaceZ'

In [132]:
emote.loc[emote['modified_emote_type'].apply(lambda x : x.startswith("BetterTTV")), 'modified_emote_type'] = 'BetterTTV'

In [499]:
grumps = pd.merge(emote, emote_channel_df, left_index=True, right_on='name').groupby(['channel', 'modified_emote_type']).sum()

In [504]:
emote.fedmyster_count.sum()

70581.0

In [448]:
def combine_series(v1, v2):
    l = not np.isnan(v1)
    r = not np.isnan(v2)
    
    if l and r:
        return v1 + v2
    elif l:
        return v1
    else:
        return v2

In [489]:
grumps['fedmyster_count']['fedmyster'].combine(grumps['fedmyster_count']['global'], combine_series)

modified_emote_type
BetterTTV        11209.0
FrankerFaceZ     14774.0
Global Twitch     5722.0
Tier 1           37379.0
Tier 2               0.0
Tier 3             367.0
Name: fedmyster_count, dtype: float64

In [478]:
grumpy_grumps.append(pd.Series(grumps['fedmyster_count'].sum() - grumpy_grumps.sum(), index=['Other']))

BetterTTV        11209.0
FrankerFaceZ     14774.0
Global Twitch     5722.0
Tier 1           37379.0
Tier 2               0.0
Tier 3             367.0
Other             1130.0
dtype: float64

In [492]:
series = []
channels = df.channel.unique()
for c in channels:
    grumpy_grumps = grumps[f'{c}_count'][c].combine(grumps[f'{c}_count']['global'], combine_series)
    grumpy_grumps = grumpy_grumps.append(pd.Series(grumps[f'{c}_count'].sum() - grumpy_grumps.sum(), index=['Other Channel Emotes']))
    top_index = [c] * len(grumpy_grumps.index)
    multi_index = pd.MultiIndex.from_tuples(list(zip(top_index, grumpy_grumps.index)))
    series.append(pd.Series(grumpy_grumps.values, index=multi_index))
pd.concat(series)

fedmyster   BetterTTV               11209.0
            FrankerFaceZ            14774.0
            Global Twitch            5722.0
            Tier 1                  37379.0
            Tier 2                      0.0
            Tier 3                    367.0
            Other Channel Emotes     1130.0
lilypichu   BetterTTV                1493.0
            FrankerFaceZ              138.0
            Global Twitch            7260.0
            Tier 1                   7568.0
            Tier 2                      5.0
            Tier 3                      0.0
            Other Channel Emotes     6145.0
pokimane    BetterTTV                2667.0
            FrankerFaceZ              888.0
            Global Twitch           12604.0
            Tier 1                   3235.0
            Tier 2                      0.0
            Tier 3                     23.0
            Other Channel Emotes    10067.0
scarra      BetterTTV                 706.0
            FrankerFaceZ        